# Transformers

## 🔎 The Problem with RNNs and LSTMs
- **Sequential bottleneck**: RNNs/LSTMs read one token at a time → slow, hard to parallelize.  
- **Long-term dependencies**: Struggle to capture relationships between distant tokens (even with LSTMs’ gates).

**Transformers (2017, _Attention is All You Need_)** solve both the issues. **Transformer** is a deep learning neural network architecture introduced in 2017 paper.

---

Models built using Transformers are for:
- voice-to-text (takes audio and generates transcript)
- text-to-voice (transcript to audio)
- text-to-image (DALL-E, Mid-Journey)
- machine translation
- summerization

**Reference**: https://poloclub.github.io/transformer-explainer/

https://prvnsmpth.github.io/animated-transformer/

## Transformer Architecture - Simplified
Original transformer architecture was developed for machine translation task translating enligsh text to german and french.

![transformer](https://media.geeksforgeeks.org/wp-content/uploads/20240729154418/Transformer.png)

Before a Transformer (encoder, decoder, or encoder–decoder) can process text, the input must be converted from **raw text** into **numerical matrices**.  
This happens in three main stages: **Tokenization**, **Embedding**, and **Positional Encoding**.

We'll use the same example throughout:

> **Example Sentence:**  
> "Harry Potter was a highly unusual boy ... least favorite teacher, professor ??"

---

## 🧠 Transformer Input Processing (Tokenization → Embedding → Positional Encoding)

### 🔹 Step 1: Tokenization

The first step is to convert human-readable text into smaller, machine-readable chunks called **tokens**.

Transformers use **subword tokenization** (e.g., **Byte Pair Encoding – BPE**).  
Tokens can be entire words, subwords, or even punctuation and whitespace.

| Stage | Representation | Description |
|--------|----------------|--------------|
| Input Text | "Harry Potter was a highly unusual boy ... least favorite teacher, professor ??" | Original sentence |
| Tokens | ["Harry", " Potter", " was", " a", " highly", " unusual", " boy", "...", " least", " favorite", " teacher", ",", " professor", " ??"] | Subword tokens |
| Token IDs | [11112, 23450, 312, 234, 1222, 5431, 2390, 891, 7654, 9876, 2343, 11, 4567, 98] | Integer IDs from vocabulary |

**Key facts:**
- GPT-3’s vocabulary dictionary has **50,257 unique tokens**.  
- Each token ID is just an **index** into a large embedding table.

---

### 🔹 Step 2: Embedding

Each token ID is converted into a **high-dimensional vector** through an **embedding lookup**.  
For example, GPT-3 175B uses an **embedding dimension of 12,288**. Each vector is represented as matrix of [1 x 12288]

For our 14-token input:
- Each token (e.g., `"Harry"`) becomes a **12,288-dimensional vector**.
- The entire sequence becomes a **matrix** with **14 rows** (one per token) and **12,288 columns** (features per token).

| Token | Token ID | Embedding Vector Shape |
|--------|-----------|------------------------|
| "Harry" | 11112 | [1 × 12,288] |
| "Potter" | 23450 | [1 × 12,288] |
| ... | ... | ... |
| "??" | 98 | [1 × 12,288] |

So, the final embedding matrix is of [14 tokens x 12288] dimensions.

### 🔹 Step 3: Positional Encoding
Transformers have **no inherent notion of word order**.  
To fix this, a **positional encoding vector** is added to each token’s embedding to indicate its **position in the sequence**.

Formally:

\[
Z = E + P
\]

Where:  
- \( E \): embedding matrix `[sequence_length × embedding_dim]`  
- \( P \): positional encoding matrix `[sequence_length × embedding_dim]`  
- \( Z \): final input matrix passed into the Transformer layers

For our example:

| Token | Position | Embedding (E) | Positional Vector (P) | Combined (Z = E + P) |
|--------|-----------|----------------|------------------------|-----------------------|
| "Harry" | 0 | [1 × 12,288] | [1 × 12,288] | [1 × 12,288] |
| "Potter" | 1 | [1 × 12,288] | [1 × 12,288] | [1 × 12,288] |
| ... | ... | ... | ... | ... |

Final matrix shape:
[14 × 12,288]


If this input is batched with others, the model may **pad** all sequences to the same length (e.g., GPT-3’s context window of **2048 tokens**): [2048 × 12,288]

GPT-3 model can take 2048 tokens or words in one go.

---

### 🔹 Step 4: Feeding into the Transformer

After combining embeddings and positional encodings, the resulting matrix \( Z \) enters the Transformer stack.  
Here’s how it differs depending on the architecture:

| Model Type | Next Step | Flow |
|-------------|------------|------|
| **Encoder-only** (e.g., BERT) | \( Z \) → Encoder layers (self-attention) | Used for classification or understanding tasks |
| **Decoder-only** (e.g., GPT-3) | \( Z \) → Decoder layers (masked self-attention) | Used for text generation |
| **Encoder–Decoder** (e.g., T5, original Transformer) | Encoder processes input → Decoder attends to encoder output | Used for translation, summarization, etc. |

---

## 🧩 Transformer Encoder Layers (Self-Attention)

After tokenization, embedding, and positional encoding, our input is now a matrix of shape:

```
[sequence_length × hidden_size]
```

For our example (**14 tokens** and **12,288 dimensions**):

```
Input Matrix: [14 × 12,288]
```

This matrix passes through multiple **encoder layers** (12 for BERT Base, 24 for BERT Large, etc.).
Each encoder layer has two key sublayers:

1. **Multi-Head Self-Attention**
2. **Feed-Forward Network (FFN)**

---

### 1️⃣ Self-Attention — “What should I pay attention to?”

Each encoder layer allows every token to **interact with all other tokens** in the same sequence.

#### Step 1: Linear Projections

For each token vector, the model computes three new representations using learned weight matrices:

* **Query (Q):** What am I looking for?
* **Key (K):** What information do I have?
* **Value (V):** The actual information to share

Each has a shape of `[sequence_length × d_k]`, e.g.:

```
[14 × 64] per attention head
```

#### Step 2: Scaled Dot-Product Attention

Attention scores are calculated as:

$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$

This mechanism helps each token “attend” to others based on relevance.
For example, **“Harry”** might attend strongly to **“Potter”**.

#### Step 3: Multi-Head Attention

Instead of one attention process, the model uses **multiple heads** (e.g., 96 in GPT-3).
Each head learns different relationships (like subject–verb or entity–reference).
All head outputs are concatenated and linearly projected back into a single matrix:

```
[14 × 12,288]
```

---

### 2️⃣ Add & Normalize

The attention output is added back to the original input (**residual connection**) and normalized:

$
\text{Output}_1 = \text{LayerNorm}(X + \text{SelfAttention}(X))
$

This stabilizes training and preserves the original information.

---

### 3️⃣ Feed-Forward Network (FFN)

Each token vector is passed through a small two-layer neural network:

$
\text{FFN}(x) = \text{ReLU}(xW_1 + b_1)W_2 + b_2
$

This expands the dimensionality (e.g., 12,288 → 49,152 → 12,288) to introduce non-linear transformations.

A second **Add & Normalize** follows:

$
\text{Output}_2 = \text{LayerNorm}(\text{Output}_1 + \text{FFN}(\text{Output}_1))
$

---

### 4️⃣ Final Encoder Output

After stacking multiple encoder layers, we get the **final contextual representation** for each token.

* **Shape remains:** `[14 × 12,288]`
* **But now:** each token vector is **context-aware** — e.g., “Harry” knows it relates to “Potter,” not “teacher.”

---

### 📊 Summary Table

| Stage            | Input Shape   | Operation                | Output Shape  | Description                  |
| ---------------- | ------------- | ------------------------ | ------------- | ---------------------------- |
| Input Embeddings | [14 × 12,288] | —                        | [14 × 12,288] | Token + positional encoding  |
| Self-Attention   | [14 × 12,288] | Multi-head attention     | [14 × 12,288] | Tokens attend to one another |
| Add & Norm       | [14 × 12,288] | Residual + normalization | [14 × 12,288] | Stabilize learning           |
| Feed-Forward     | [14 × 12,288] | Linear → ReLU → Linear   | [14 × 12,288] | Token-wise transformation    |
| Add & Norm       | [14 × 12,288] | Residual + normalization | [14 × 12,288] | Final encoder output         |


## 🧩 Transformer Decoder Layers (with Encoder Input for Translation)

Assume we are performing a **translation task**, and the decoder receives **encoder outputs** from the input sentence:

> Original Sentence: "Harry Potter was a highly unusual boy ... least favorite teacher, professor ??"

The **encoder** has already processed this input and produced a **contextualized representation** for each token:

```
Encoder Output Matrix: [14 × 12,288]
```

This matrix will serve as the **key and value** for the decoder's cross-attention layers.

---

### 1️⃣ Decoder Input

The decoder takes its own target sequence as input, e.g., a partially generated translation:

```
Decoder Input Tokens: "Harry Potter était un garçon ... professeur préféré ??"
Token IDs → Embeddings → Positional Encoding
Decoder Input Matrix: [14 × 12,288]
```

---

### 2️⃣ Masked Self-Attention

* The decoder computes **queries, keys, and values** from its own input.
* **Masking** ensures that token *i* only attends to previous tokens `0` to `i` (not future tokens).
* This produces **context-aware decoder vectors** that respect autoregressive generation.

```
Shape: [14 × 12,288]
```

---

### 3️⃣ Cross-Attention (Attending to Encoder Outputs)

* The decoder's **queries (Q)** come from the masked self-attention output.
* The **keys (K)** and **values (V)** come from the **encoder output** `[14 × 12,288]`.

[
ext{CrossAttention}(Q_{dec}, K_{enc}, V_{enc}) = 	ext{softmax}\left(rac{Q_{dec} K_{enc}^T}{\sqrt{d_k}}
ight) V_{enc}
]

* This step allows each decoder token to **attend to relevant parts of the original input sentence**, e.g., “Harry” in French aligns with “Harry” in English.
* Output shape: `[14 × 12,288]`

---

### 4️⃣ Feed-Forward Network (FFN)

* Each token vector is independently passed through a **2-layer feed-forward network** with residual connections and layer normalization.
* Output shape remains `[14 × 12,288]`.

[
ext{FFN}(x) = 	ext{ReLU}(xW_1 + b_1) W_2 + b_2
]

---

### 5️⃣ Final Decoder Output

* After multiple decoder layers (e.g., 12–96), each token vector encodes:

  * Its own past tokens (**masked self-attention**)
  * Relevant information from the **encoder output** (**cross-attention**)

* These vectors are then projected through a **linear + softmax layer** to predict the **next token probabilities** in the target language.

---

### 📊 Summary Table (Decoder with Encoder Input)

| Stage                            | Input Shape                                       | Operation                | Output Shape  | Description                                     |
| -------------------------------- | ------------------------------------------------- | ------------------------ | ------------- | ----------------------------------------------- |
| Embeddings + Positional Encoding | [14 × 12,288]                                     | —                        | [14 × 12,288] | Decoder input tokens                            |
| Masked Self-Attention            | [14 × 12,288]                                     | Multi-head attention     | [14 × 12,288] | Attend to previous tokens only                  |
| Add & Norm                       | [14 × 12,288]                                     | Residual + normalization | [14 × 12,288] | Stabilize learning                              |
| Cross-Attention                  | [14 × 12,288] (decoder) + [14 × 12,288] (encoder) | Multi-head attention     | [14 × 12,288] | Attend to encoder output (source sentence)      |
| Add & Norm                       | [14 × 12,288]                                     | Residual + normalization | [14 × 12,288] | Stabilize learning                              |
| Feed-Forward                     | [14 × 12,288]                                     | Linear → ReLU → Linear   | [14 × 12,288] | Token-wise transformation                       |
| Add & Norm                       | [14 × 12,288]                                     | Residual + normalization | [14 × 12,288] | Final decoder output ready for token prediction |

---

> **In summary:**
> For translation tasks, the decoder layers combine **masked self-attention** and **cross-attention** to ensure that each generated token is informed by:
>
> 1. Its previous tokens in the target sequence
> 2. The contextual representation of the input sequence from the encoder

```
```




# How GPT-3 Works: The Magic Behind the Next Word

Have you ever wondered what's happening under the hood when a tool like GPT-3 or ChatGPT instantly generates a coherent and human-like response? The process, while complex, can be broken down into a series of logical steps. At its heart, GPT-3 is an incredibly powerful prediction machine, designed to do one thing exceptionally well: predict the next word in a sequence. This is the magic behind every poem, email, or line of code it writes.

Here’s a step-by-step breakdown of how a model like GPT-3 predicts the missing word "Snape" in the sentence "Harry Potter was a highly unusual boy ... least favorite teacher, professor ??"

## Step 1: Tokenization (BPE)
The very first step is to translate the human-readable text into a format the model can understand. This process is called tokenization. GPT-3 uses Byte-Pair Encoding (BPE), which breaks down text into smaller units called tokens. These aren't just whole words; they can be parts of words or common symbols, making the model highly efficient.

**Input Sentence:** "Harry Potter was a highly unusual boy ... least favorite teacher, professor ??"

**Output Tokens:** ["Harry", " Potter", " was", " a", " highly", " unusual", " boy", "...", " least", " favorite", " teacher", ",", " professor", " ??"]

**Output IDs:** [11112, 23450, 312, 234, 1222, 5431, 2390, 891, 7654, 9876, 2343, 11, 4567, 98]

Each token is then mapped to a unique integer ID from the model's large vocabulary of 50,257 tokens. This transforms the text into a sequence of numbers, which is the actual input for the next step.

![transformer](https://media.geeksforgeeks.org/wp-content/uploads/20240729154418/Transformer.png)

## Step 2: Embedding + Positional Encoding
Next, these token IDs are converted into a more meaningful representation: a high-dimensional vector. This process is called embedding. Each token ID is looked up in a massive table, which maps it to a vector of 12,288 dimensions. This vector captures the token's semantic meaning.

To tell the model about the order of the words, a positional encoding vector is added to each token's embedding. This unique vector tells the model the exact position of each token in the sequence (e.g., "Harry" is at position 0, "Potter" at position 1, and so on). Together, the embedding and positional encoding create a single, powerful vector that encodes both the token's meaning and its position.

The model's context window is the maximum number of tokens it can process at once (e.g., 2048 tokens for some versions of GPT-3). For computational efficiency, all input sequences are padded to this fixed length.

**Input IDs:** [11112, 23450, 312, ...]

**Output Matrix:** A matrix where each row is a token's vector. The shape would be [14 tokens × 12,288 dimensions], which is then padded to [2048 × 12,288] for batch processing.

## Step 3: The Transformer Decoder Blocks (×96)
The core of GPT-3 is a stack of 96 Transformer decoder blocks. Each block refines the token vectors by applying two key mechanisms that allow the model to understand the relationships between words.

### A. Masked Multi-Head Self-Attention
This is where the model determines which words are most relevant to each other to establish context. For example, when processing the token "professor," the model's query for context finds a high similarity with the keys of "teacher," "favorite," and "Harry Potter." This tells the model what information is most important for making its prediction.

The multi-head part means the model does this not once, but in parallel across multiple "heads," each of which learns to focus on a different aspect of the sentence (e.g., one head might focus on grammatical relationships, while another focuses on character names). The masked part ensures the model cannot "cheat" by looking at the future token it's trying to predict.

#### Here's how it works at a matrix level:

- **Creating the Query, Key, and Value Matrices:** The input is a matrix of token embeddings, X, with a shape of [T × d_model]. For each attention head, the model learns three separate weight matrices (W_Q, W_K, and W_V) to project X into three new matrices: Q (Query), K (Key), and V (Value). The dimensions of these new matrices are d_k and d_v, which are typically smaller than d_model.

- **Query Matrix (Q):** Represents what each token is "looking for." Q = XW_Q. Output shape: [T × d_k]

- **Key Matrix (K):** Represents what each token "contains" to be found. K = XW_K. Output shape: [T × d_k]

- **Value Matrix (V):** Contains the actual content information for each token. V = XW_V. Output shape: [T × d_v]

- **Calculating Attention Scores:** The dot product of the Query and Key matrices is calculated to find the similarity between every pair of tokens. This creates a matrix of attention scores.

**Attention Scores Matrix:** AttentionScores = QK^T. Output shape: [T × T].

This matrix is the heart of the attention mechanism. Each element (i, j) represents how much token i should "pay attention" to token j. In a decoder, a mask is applied to this matrix to prevent tokens from looking at future tokens.

The scores are then scaled by the square root of d_k and a softmax function is applied, normalizing the scores into a probability distribution.

- **Generating the Output Matrix:** The normalized attention weights are then multiplied by the Value matrix V. This creates a weighted sum of the values, where the weights are the attention scores.

### B. Feed-Forward Network (FFN)
After attention, each token's vector is processed by a two-layer neural network. This network adds abstract features and non-linearities, helping the model learn more complex patterns, such as recognizing that the current context is likely a slot for a character's name.

**Input Matrix:** [14 × 12,288] (from Step 2)

**Output Matrix:** A new [14 × 12,288] matrix where each vector is context-aware.

- Row 1: [0.34, 0.67, 0.12, ...] (contextualized vector for "Harry")
- Row 2: [0.88, 0.45, 0.91, ...] (contextualized vector for "Potter")
- ...
- Row 14: [0.91, 0.56, 0.23, ...] (contextualized vector for "??")

**Output Shape:** [14 × 12,288]

## Step 4: Final Linear Layer + Softmax
The final step is to use the deeply contextualized vector of the last token, "??", to make a prediction. This vector is passed through a final linear layer that projects it back to the size of the vocabulary (50,257). The output of this layer is a list of scores, called logits.

A Softmax function is then applied to these logits, converting them into a probability distribution where the probabilities of all possible next tokens sum to 1. Based on the context "Harry Potter... least favorite teacher, professor," the model would assign the highest probabilities to the most likely candidates:

**Input Vector:** [1 × 12,288] (the last row from the output of Step 3)

**Logits Vector:** [1 × 50,257]

[..., 0.1, 0.5, 0.9, 0.05, ...] (before Softmax)

**Softmax Output (Probabilities):**
- P(" Snape"): 0.72
- P(" Dumbledore"): 0.09
- P(" McGonagall"): 0.05

**Output Shape:** [1 × 50,257]

## Step 5: Autoregressive Generation
Finally, the model selects the token with the highest probability ("Snape"), adds it to the sequence, and feeds this new, longer sequence back into the model. This process repeats, predicting one word at a time, until a stop token (like a period) is generated or a length limit is reached. This is how GPT-3 creates a complete and continuous output, one word at a time.


---

# How GPT-3 Reaches 175 Billion Parameters

GPT-3’s massive size (175B parameters) comes from stacking many Transformer decoder blocks and the associated embedding layers. Here’s how those parameters add up:

## 1. Embedding Layer
- **Vocabulary size:** 50,257 tokens  
- **Embedding dimension (`d_model`):** 12,288  
- **Parameters:** 50,257 × 12,288 ≈ **617M**  

This is the word embedding table (also reused for the output layer due to weight tying).

## 2. Transformer Decoder Blocks (×96)
Each block has self-attention + feed-forward + layer norms.

### A. Multi-Head Self-Attention
- Query, Key, Value projections: 3 × (12,288 × 12,288) ≈ 453M  
- Output projection: 12,288 × 12,288 ≈ 151M  
- **Total per attention:** ≈ 604M  

### B. Feed-Forward Network (FFN)
- Expands hidden size 4×: 12,288 → 49,152 → 12,288  
- First linear: 12,288 × 49,152 ≈ 603M  
- Second linear: 49,152 × 12,288 ≈ 603M  
- **Total per FFN:** ≈ 1.2B  

### C. Layer Norms and Biases
Small compared to others (millions).

**Total per block:** ≈ 1.8B parameters  
**Across 96 blocks:** 96 × 1.8B ≈ **173B**

## 3. Final Layer Norm + Output Head
- Small, but output head ties with embeddings (~617M).

## ✅ Grand Total
- Embeddings: ~0.6B  
- Transformer blocks: ~173B  
- Misc + tied weights: ~1B  
- **≈ 175B parameters**

---

👉 **Key insight:** Most parameters (~99%) are in the **feed-forward networks**, not embeddings or attention.


# How BERT Works: The Magic Behind Understanding Language

Have you ever wondered how a model like BERT can understand the meaning
of a sentence, answer questions, or detect sentiment? Unlike GPT, which
is designed to *predict the next word*, BERT is built to deeply
understand the **entire sentence in context**. This makes it especially
powerful for tasks like search engines, question answering, and
sentiment analysis.

At its heart, BERT is not trying to write the next word---it's trying to
**grasp meaning** by looking at all the words at once. Let's break down
how BERT works with an example.

Suppose we want BERT to figure out the missing word in this sentence:

*"Harry Potter was a highly unusual boy. His least favorite teacher was
Professor \[MASK\]."*

We want BERT to predict the masked word ("Snape") based on the entire
context.

------------------------------------------------------------------------

## Step 1: Tokenization (WordPiece)

Just like GPT, BERT starts by breaking the text into smaller pieces
called tokens. BERT uses **WordPiece tokenization**, which breaks rare
words into subwords.

**Input Sentence:**\
"Harry Potter was a highly unusual boy. His least favorite teacher was
Professor \[MASK\]."

**Output Tokens:**\
\["\[CLS\]", "Harry", "Potter", "was", "a", "highly", "unusual", "boy",
".", "His", "least", "favorite", "teacher", "was", "Professor",
"\[MASK\]", ".", "\[SEP\]"\]

-   `[CLS]` is a special classification token added at the start.\
-   `[SEP]` marks the end of a sentence.\
-   `[MASK]` is a placeholder telling BERT to guess this word.

Each token is then mapped to an ID, just like GPT.

------------------------------------------------------------------------

## Step 2: Embedding + Positional Encoding

Every token ID is converted into a high-dimensional vector. But BERT
embeddings are richer than GPT's---they include three kinds of
information:

1.  **Token Embedding:** The meaning of the token (e.g., "Harry").\
2.  **Segment Embedding:** Whether the token belongs to sentence A or B
    (for tasks like Question + Answer).\
3.  **Position Embedding:** The token's position in the sequence (so
    BERT knows word order).

These vectors are summed up into one embedding for each token, forming
the input matrix:

**Input IDs → Embedding Matrix:** Shape = \[T tokens × d_model\]\
For BERT-Base, `d_model = 768`; for BERT-Large, `d_model = 1024`.

------------------------------------------------------------------------

## Step 3: Transformer Encoder Blocks (×12 or ×24)

Here's where BERT differs most from GPT. Instead of a **decoder-only
stack** like GPT, BERT uses **encoder-only Transformer blocks**.

### A. Bidirectional Self-Attention

BERT's attention mechanism looks at *all* words at once, both left and
right.

-   When guessing `[MASK] = "Snape"`, BERT doesn't just look left
    ("Professor") or right ("."), it also considers context like "least
    favorite teacher" earlier in the sentence.\
-   This bidirectional nature allows it to truly understand context,
    unlike GPT, which can only look backward.

Technically, it works the same way as GPT's attention:\
- Each token generates a Query (Q), Key (K), and Value (V).\
- Attention scores = QK\^T, normalized by softmax.\
- Each token becomes a **weighted combination of all other tokens**.

But unlike GPT, there's **no masking**---BERT is allowed to see the full
sentence.

### B. Feed-Forward Networks

After attention, each token's vector goes through a small neural network
that enriches its representation.

Output: Contextualized embeddings, where each token now "knows" its
meaning relative to the full sentence.

------------------------------------------------------------------------

## Step 4: The Output Head (Masked Language Modeling)

Now comes the special trick: BERT was trained with the **Masked Language
Model (MLM)** objective.

-   Some tokens (like "Snape") are replaced with `[MASK]`.\
-   BERT must predict the original word using all surrounding context.

For our example:

**Input Vector (for \[MASK\]):** \[1 × 768\]\
**Logits Vector:** \[1 × 30,000\] (BERT's vocab size)

After applying **Softmax**, BERT assigns probabilities:

-   P("Snape") = 0.83\
-   P("Dumbledore") = 0.06\
-   P("McGonagall") = 0.04

BERT picks **"Snape"**, which makes sense in context.

------------------------------------------------------------------------

## Step 5: Fine-Tuning for Specific Tasks

Unlike GPT, which is mostly used for text generation, BERT is designed
for **fine-tuning**. After pretraining on billions of words, BERT can be
adapted for:

-   **Sentiment Analysis:** Feed in a sentence → use `[CLS]` embedding →
    classify as Positive/Negative.\
-   **Question Answering:** Feed in a question + passage → BERT
    highlights the answer span.\
-   **Named Entity Recognition:** Classify each token (e.g., "Harry" →
    PERSON).

This versatility is why BERT became the foundation for so many modern
NLP models.

------------------------------------------------------------------------

# How BERT Reaches 340 Million Parameters

BERT comes in two main sizes:

-   **BERT-Base:** 110M parameters (12 layers, 12 heads, hidden size
    768)\
-   **BERT-Large:** 340M parameters (24 layers, 16 heads, hidden size
    1024)

Where do these parameters come from?

1.  **Embedding Layer:**
    -   Vocab size ≈ 30,000\
    -   Hidden size = 768 (base) or 1024 (large)\
    -   Params ≈ 23M (base), 30M (large)
2.  **Transformer Encoder Blocks:**
    -   Each block = Multi-Head Attention + FFN\
    -   For base: \~7M per block × 12 = \~85M\
    -   For large: \~13M per block × 24 = \~310M
3.  **Output Head:**
    -   Small compared to the rest.

**Grand Total** = 110M (base) or 340M (large)





👉 **Key Insight:**\
- **GPT = Autoregressive (predict next word, one direction)**\
- **BERT = Autoencoding (fill in blanks, bidirectional)**

This is why GPT shines at writing stories, while BERT excels at
understanding text.